In [2]:
from mxnet import nd
from mxnet.contrib import text 

text.embedding.get_pretrained_file_names().keys()

dict_keys(['glove', 'fasttext'])

In [3]:
print(text.embedding.get_pretrained_file_names('glove'))

['glove.42B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.840B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt']


In [4]:
glove_6b50d = text.embedding.create(
    'glove', pretrained_file_name='glove.6B.50d.txt')

In [5]:
len(glove_6b50d)

400001

In [6]:
glove_6b50d.token_to_idx['beautiful']

3367

In [7]:
glove_6b50d.token_to_idx['standard']

1225

In [8]:
def knn(W, x, k):
    w_l2 = nd.sum(W*W, axis=1) + 1e-9 # for numerical stability
    x_l2 = nd.sum(x*x)
    cos = nd.dot(W, x.reshape((-1,)))/(w_l2.sqrt()*x_l2.sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [14]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, 
                    embed.get_vecs_by_tokens([query_token]),k+1)
    for i,c in zip(topk[1:],cos[1:]):
        print('cosine sim=%.3f: %s' % (c, embed.idx_to_token[i]))

In [15]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [16]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [17]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


In [18]:
get_similar_tokens('standard', 3, glove_6b50d)

cosine sim=0.797: standards
cosine sim=0.776: type
cosine sim=0.775: uses


In [19]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c])
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[topk[0]]

In [20]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [21]:
get_analogy('king', 'queen', 'son', glove_6b50d)

'daughter'

In [22]:
get_analogy('king', 'queen', 'prince', glove_6b50d)

'princess'